In [1]:
# Dependencies
import pandas as pd
import numpy as np

In [5]:
# Create a file path
# Encoding from: https://stackoverflow.com/a/48067785/23471668
raw_df = pd.read_csv("../Data/monroe county car crach 2003-2015.csv", encoding='windows-1252')
# Read in the data.
raw_df.head()

,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude
0,2015,1,5,Weekday,0.0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874
1,2015,1,6,Weekday,1500.0,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848
2,2015,1,6,Weekend,2300.0,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890
3,2015,1,7,Weekend,900.0,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956
4,2015,1,7,Weekend,1100.0,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625


In [6]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53943 entries, 0 to 53942
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               53943 non-null  int64  
 1   Month              53943 non-null  int64  
 2   Day                53943 non-null  int64  
 3   Weekend?           53875 non-null  object 
 4   Hour               53718 non-null  float64
 5   Collision Type     53937 non-null  object 
 6   Injury Type        53943 non-null  object 
 7   Primary Factor     52822 non-null  object 
 8   Reported_Location  53908 non-null  object 
 9   Latitude           53913 non-null  float64
 10  Longitude          53913 non-null  float64
dtypes: float64(3), int64(3), object(5)
memory usage: 4.5+ MB


In [16]:
# Remove nulls
new_df=raw_df.dropna(how="any").reset_index()
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52582 entries, 0 to 52581
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              52582 non-null  int64  
 1   Year               52582 non-null  int64  
 2   Month              52582 non-null  int64  
 3   Day                52582 non-null  int64  
 4   Weekend?           52582 non-null  object 
 5   Hour               52582 non-null  float64
 6   Collision Type     52582 non-null  object 
 7   Injury Type        52582 non-null  object 
 8   Primary Factor     52582 non-null  object 
 9   Reported_Location  52582 non-null  object 
 10  Latitude           52582 non-null  float64
 11  Longitude          52582 non-null  float64
dtypes: float64(3), int64(4), object(5)
memory usage: 4.8+ MB


In [17]:
# Removing the Weekend column because it looks to include times
df=new_df.drop("Weekend?", axis=1)
df.head()

,index,Year,Month,Day,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude
0,0,2015,1,5,0.0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874
1,1,2015,1,6,1500.0,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848
2,2,2015,1,6,2300.0,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890
3,3,2015,1,7,900.0,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956
4,4,2015,1,7,1100.0,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625


In [18]:
# Allows for dot notation
df.columns = [x.lower().replace(" ", "_") for x in df.columns]
df.head()

,index,year,month,day,hour,collision_type,injury_type,primary_factor,reported_location,latitude,longitude
0,0,2015,1,5,0.0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874
1,1,2015,1,6,1500.0,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848
2,2,2015,1,6,2300.0,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890
3,3,2015,1,7,900.0,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956
4,4,2015,1,7,1100.0,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625


In [26]:
# Combining similar primary factors
df["primary_factor"] = df.primary_factor.replace({"OTHER (VEHICLE) - EXPLAIN IN NARRATIVE": "OTHER",
                                                  "OTHER (DRIVER) - EXPLAIN IN NARRATIVE": "OTHER",
                                                  "OTHER (ENVIRONMENTAL) - EXPLAIN IN NARR": "OTHER",
                                                  "VIOLATION OF LICENSE RESTRICTION": "OTHER",
                                                  "NONE (ENVIRONMENTAL)": "NONE",
                                                  "NONE (VEHICLE)": "NONE",
                                                  "NONE (DRIVER)": "NONE",
                                                  "RAN OFF ROAD RIGHT": "RAN OFF ROAD",
                                                  "RAN OFF ROAD LEFT": "RAN OFF ROAD",
                                                  "PASSENGER DISTRACTION": "DISTRACTED DRIVING",
                                                  "DRIVER DISTRACTED - EXPLAIN IN NARRATIVE": "DISTRACTED DRIVING",
                                                  "DRIVER ASLEEP OR FATIGUED": "DISTRACTED DRIVING",
                                                  "CELL PHONE USAGE": "DISTRACTED DRIVING",
                                                  "ALCOHOLIC BEVERAGES": "DRIVING UNDER THE INFLUENCE",
                                                  "PRESCRIPTION DRUGS": "DRIVING UNDER THE INFLUENCE",
                                                  "ILLEGAL DRUGS": "DRIVING UNDER THE INFLUENCE",                                                  
                                                  "ENGINE FAILURE OR DEFECTIVE": "VEHICLE DEFECT",
                                                  "OTHER LIGHTS DEFECTIVE": "VEHICLE DEFECT",
                                                  "TOW HITCH FAILURE": "VEHICLE DEFECT",
                                                  "ACCELERATOR FAILURE OR DEFECTIVE": "VEHICLE DEFECT",
                                                  "STEERING FAILURE": "VEHICLE DEFECT",
                                                  "BRAKE FAILURE OR DEFECTIVE": "VEHICLE DEFECT",
                                                  "HEADLIGHT DEFECTIVE OR NOT ON": "VEHICLE DEFECT",
                                                  "TIRE FAILURE OR DEFECTIVE": "VEHICLE DEFECT",
                                                  "INSECURE/LEAKY LOAD": "VEHICLE DEFECT",
                                                  "OVERSIZE/OVERWEIGHT LOAD": "VEHICLE DEFECT",
                                                  "JACKKNIFING": "VEHICLE DEFECT",
                                                  "OTHER TELEMATICS IN USE": "ROAD CONDITIONS",
                                                  "LANE MARKING OBSCURED": "ROAD CONDITIONS",
                                                  "SHOULDER DEFECTIVE": "ROAD CONDITIONS",
                                                  "UTILITY WORK": "ROAD CONDITIONS",
                                                  "HOLES/RUTS IN SURFACE": "ROAD CONDITIONS",
                                                  "ROAD UNDER CONSTRUCTION": "ROAD CONDITIONS",
                                                  "ROADWAY SURFACE CONDITION": "ROAD CONDITIONS",
                                                  "ANIMAL/OBJECT IN ROADWAY": "ROAD CONDITIONS",
                                                  "SEVERE CROSSWINDS": "ROAD CONDITIONS",
                                                  "GLARE": "ROAD CONDITIONS",
                                                  "OBSTRUCTION NOT MARKED": "ROAD CONDITIONS",
                                                  "TRAFFIC CONTROL INOPERATIVE/MISSING/OBSC": "ROAD CONDITIONS"})
df["primary_factor"].value_counts()

primary_factor
FAILURE TO YIELD RIGHT OF WAY            11175
FOLLOWING TOO CLOSELY                     7345
OTHER                                     6952
UNSAFE BACKING                            5166
ROAD CONDITIONS                           3107
RAN OFF ROAD                              2979
DISREGARD SIGNAL/REG SIGN                 2201
DISTRACTED DRIVING                        2089
SPEED TOO FAST FOR WEATHER CONDITIONS     1917
IMPROPER TURNING                          1836
UNSAFE SPEED                              1492
LEFT OF CENTER                            1071
IMPROPER LANE USAGE                        978
DRIVING UNDER THE INFLUENCE                884
UNSAFE LANE MOVEMENT                       753
VEHICLE DEFECT                             665
OVERCORRECTING/OVERSTEERING                596
IMPROPER PASSING                           493
PEDESTRIAN ACTION                          292
DRIVER ILLNESS                             180
VIEW OBSTRUCTED                            17

In [29]:
# Save off csv
df.to_csv("../Data/cleaned_car_crash_data.csv", index=False)